# Homework Assignment 2: Data Prep
In this homework assignment, you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).


This assignment will have you explore the cardinalities, number of missing values, detect outliers, handle missing values and outliers, and create data quality report for original and cleaned dataset. Additionally, you will be asked to provide documentation for your functions.

## Step 1: Downloading the Data

This assignment will only be using [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB). Recall that in Homework 1 we started to construct the analytics base table for our [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM). In that assignment, we read the data from the two subdirectories, __FL__ and __NF__, of the __partition1__ direcotry. These two subdirectories represented the two classes of our target feature in the solar flare prediction problem we will be attempting to solve this semester. We then processed these samples of multivariate time series to construct descriptive features for each sample, and then placed them into our analytics base table.

In this assignment, you will be utilizing a set of extracted descriptive features much like what you were asked to construct in Homework 1. However, this dataset contains many more extracted features than you were asked to compute for Homework 1 (>800), so we need to explore the data to find data quality issues and identify ways to address these issues. Below are links to the full extracted feature dataset for partition 1 and a toy dataset to use for testing you functions.

__Note:__ Since the full dataset, and multiple copies of partially processed intermediary results, tend to take up a bit of space, you can use the toy dataset to implement and test your code. You may need to edit the data to fully test each of the requirements, but that is left as an exercise for the student. The full partition dataset is only included for those who wish to work with it once they have their code implemented. 

- [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv)
- [Toy Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_partition1ExtractedFeatures.csv)

Now that you have the extracted features csv files, you will load that data into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method.  


In [594]:
import os
import pandas as pd
from pandas import DataFrame 
import numpy as np

### Read the dataset

In [595]:
data_dir = '/Users/carltonbrown/Downloads/partition1/'
data_file = 'partition1ExtractedFeatures.csv'
#/Users/carltonbrown/Downloads/partition1/partition1ExtractedFeatures.csv.download

In [596]:
abt = pd.read_csv(os.path.join(data_dir, data_file), index_col=0)

### Q1 (20 points)

Write a function to extract the various pieces of a data quality report, for a specific attribute, and return a dataframe with this information.

 * 'Feature Name': Contains the time series statistical feature name
 
 * 'Cardinality': Contains the count of unique values for the feature
            
 * 'Non-null Count': Contains the number of non-null entries for the feature
            
 * 'Null Count': Contains the number of null or missing entries for the feature
            
 * 'Min': Contains the minimum value of the feature (Without considering the null or nan value)
 
 * '25th': Contains the first quartile (25%) of the feature values (Without considering the null/nan value)
 
 * 'Mean': Contains the mean of the feature values (Without considering the null/nan value)
 
 * '50th': Contains the median of the feature values (Without considering the null/nan value)
            
 * '75th': Contains the third quartile (75%) of the feature values (Without considering the null/nan value)
 
 * 'Max': Contains the maximum value of the feature (Without considering the null/nan value),
            
 * 'Std. Dev': Contains the standard deviation of the feature (Without considering the null/nan value)
 
In addition to the values above, you should identify the number of upper and lower outliers using the $val < Q1-1.5IQR$ and $val > Q3+1.5IQR$ outlier identification method. These added features should be called `Outlier Count Low` and `Outliers Count High` respectively.


 
 Some useful functions for this can be found at:
 
 * [Numpy.percentile](https://numpy.org/doc/stable/reference/generated/numpy.percentile.html)
 
 * [pandas.isna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isna.html)
 
 * [Numpy.mean](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)
 
 * [Numpy.std](https://numpy.org/doc/stable/reference/generated/numpy.std.html)
 

In [597]:
def calc_summary_for(feature_name:str, data:DataFrame) -> DataFrame:
    """
    this function extracts the various pieces of a data quality report, for a specific attribute, and returns a dataframe with this information.
    
    Parameters
    ----------

    data : DataFrame
        an analytical base table dataframe or general dataframe used to create a the quality report to investigate the data
    Output
    ----------
    returns various pieces of a data quality report, for a specific attribute in a dataframe.
    """
    summary_feature_names = ['Feature Name', 'Cardinality', 'Non-null Count', 'Null Count', 'Min', '25th', 'Mean', 
                             '50th', '75th', 'Max', 'Std. Dev','Outlier Count Low', 'Outlier Count High']
    #print(abt)
    QRDF = pd.DataFrame(columns =summary_feature_names)
    NewData =dict.fromkeys(summary_feature_names)
    
    
    NewData['Feature Name']=feature_name
    NewData['Cardinality']=abt[feature_name].nunique()
    NewData['Non-null Count']=abt[feature_name].count()
    NewData['Null Count']=abt[feature_name].isna().sum()
    NewData['Min']=abt[feature_name].to_numpy().min()
    NewData['25th']=np.percentile(abt[feature_name].to_numpy(), 25)
    NewData['Mean']=np.mean(abt[feature_name].to_numpy())
    NewData['50th']=np.percentile(abt[feature_name].to_numpy(), 50)
    NewData['75th']=np.percentile(abt[feature_name].to_numpy(), 75)
    NewData['Max']=abt[feature_name].to_numpy().max()
    NewData['Std. Dev']=np.std(abt[feature_name].to_numpy())
    q1 = abt[feature_name].quantile(0.25)
    q3 = abt[feature_name].quantile(0.75)
    iqr =  q3 - q1
    NewData['Outlier Count Low']=abt[feature_name].lt(q1-(1.5 *iqr)).sum()
    NewData['Outlier Count High']=abt[feature_name].gt(q3+(1.5 *iqr)).sum()
    values = NewData.values()
    QRDF.loc[len(QRDF)] =list(values)
    

    

    return QRDF
    pass


In [598]:
summary_TOTUSJH_min = calc_summary_for('TOTUSJH_min', abt)
print(summary_TOTUSJH_min)

  Feature Name Cardinality Non-null Count Null Count  Min       25th  \
0  TOTUSJH_min       30165          73492          0  0.0  29.756125   

         Mean        50th        75th          Max    Std. Dev  \
0  396.697968  110.011893  418.614528  5680.232811  694.293786   

  Outlier Count Low Outlier Count High  
0                 0               9014  


### Q2 (20 points)
Using what you produced to answere Q1, you should now write a function to construct the data quality report for all of the numerical features of our dataset.  You should loop over all of the features in the analytics base table represented by the input feature dataset files from partition 1, with the exception of the first column (this is the index column if you read the file correctly), and the `id`, `lab`, `st`, and `et` columns.  

Your output from this function will be a DataFrame that has 1 row for each feature. 

In [599]:
def construct_quality_report(data:DataFrame) -> DataFrame:
    """
    constructs a data quality report for all of the numerical features in a given dataset.
    
    Parameters
    ----------
    
        
    data : DataFrame
        an analytical base table dataframe or general dataframe used to create a the quality report to investigate the data
    Output
    ----------
    returns a summary table dataframe for all of the numerical features in a given dataset.
    """
    excluded_columns = ['id', 'lab', 'st', 'et']
    
    summary_feature_names = ['Feature Name', 'Cardinality', 'Non-null Count', 'Null Count', 'Min', '25th', 'Mean', 
                             '50th', '75th', 'Max', 'Std. Dev','Outlier Count Low', 'Outlier Count High']
    QRDF = pd.DataFrame(columns =summary_feature_names)
    NewData =dict.fromkeys(summary_feature_names)
    #count
    #print(abt.columns)
    #abt.pop('lab')
    
    for x in abt.columns:
        
        if x in excluded_columns:
            continue
        NewData['Feature Name']=x
        NewData['Cardinality']=abt[x].nunique()
        NewData['Non-null Count']=abt[x].count()
        NewData['Null Count']=abt[x].isna().sum()
        NewData['Min']=abt[x].to_numpy().min()
        NewData['25th']=np.percentile(abt[x].to_numpy(), 25)
        NewData['Mean']=np.mean(abt[x].to_numpy())
        NewData['50th']=np.percentile(abt[x].to_numpy(), 50)
        NewData['75th']=np.percentile(abt[x].to_numpy(), 75)
        NewData['Max']=abt[x].to_numpy().max()
        NewData['Std. Dev']=np.std(abt[x].to_numpy())
        q1 = abt[x].quantile(0.25)
        q3 = abt[x].quantile(0.75)
        iqr =  q3 - q1
        NewData['Outlier Count Low']=abt[x].lt(q1-(1.5 *iqr)).sum()
        NewData['Outlier Count High']=abt[x].gt(q3+(1.5 *iqr)).sum()
        values = NewData.values()
        QRDF.loc[len(QRDF)] =list(values)
    
    return QRDF
  
    pass

In [600]:
summary_table = construct_quality_report(abt)
print(summary_table.shape)  # checking the dimensionality is often a useful test.
print(summary_table)

(816, 13)
                               Feature Name Cardinality Non-null Count  \
0                               TOTUSJH_min       30165          73492   
1                               TOTUSJH_max       24142          73492   
2                            TOTUSJH_median       62001          73492   
3                              TOTUSJH_mean       73492          73492   
4                            TOTUSJH_stddev       73492          73492   
..                                      ...         ...            ...   
811                      R_VALUE_last_value       33137          73492   
812  R_VALUE_slope_of_longest_mono_increase        9737          73492   
813  R_VALUE_slope_of_longest_mono_decrease       10383          73492   
814         R_VALUE_avg_mono_increase_slope       40605          73492   
815         R_VALUE_avg_mono_decrease_slope       40729          73492   

    Null Count       Min       25th        Mean        50th        75th  \
0            0  0.000000  

### Q3 (20 points)
#### Drop features with improper cardinality:
Using the quality report summary table that is returned from the function you wrote for Q2, we are now going to investigate our data. For this, you should use the table returned for the [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv) and not the toy dataset I provided for testing.

Since we are using real valued features, a majority of them shall have a cardinality close to the sample count. So, for this question, you are to write a function that takes in the summary table and the input dataset DataFrame, and drops the feature that have a cardinality less than 10. This feature should be dropped from both the data quality report summary table and from the actual input dataset DataFrame.

A useful method for this operation is:

* [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) (Make sure to use the inplace option otherwise it returns a copy)

In [601]:
def drop_low_card_data(summary_table:DataFrame, data:DataFrame) -> None:
    """
    a function that takes in the summary table and the input dataset DataFrame, and drops 
    the features that have a cardinality less than 10.
    
    Parameters
    ----------
    summary_table : DataFrame
        a summary table that contains extracted pieces of a data quality report for the partnering analytic base table
        
    data : DataFrame
        an analytical base table dataframe or general dataframe used alongside the quality report to investigate the data
    Output
    ----------
    returns a summary table dataframe after remvoing features that have a cardinality less than 10 from the dataframe.
    """
    Name = summary_table['Feature Name']
    Cardin = summary_table['Cardinality']
    
    count = 0
    
    
    for x, y in zip(Name, Cardin):
        
        if y < 10:
            #print(y)
            abt.drop(x, axis=1, inplace=True)
            continue
        count = count+1
    
    
    
    
    summary_table.drop(summary_table[summary_table['Cardinality'] < 10].index, inplace = True)
    
    pass


In [602]:
drop_low_card_data(summary_table, abt)
print(summary_table.shape)
print(summary_table)

(801, 13)
                               Feature Name Cardinality Non-null Count  \
0                               TOTUSJH_min       30165          73492   
1                               TOTUSJH_max       24142          73492   
2                            TOTUSJH_median       62001          73492   
3                              TOTUSJH_mean       73492          73492   
4                            TOTUSJH_stddev       73492          73492   
..                                      ...         ...            ...   
811                      R_VALUE_last_value       33137          73492   
812  R_VALUE_slope_of_longest_mono_increase        9737          73492   
813  R_VALUE_slope_of_longest_mono_decrease       10383          73492   
814         R_VALUE_avg_mono_increase_slope       40605          73492   
815         R_VALUE_avg_mono_decrease_slope       40729          73492   

    Null Count       Min       25th        Mean        50th        75th  \
0            0  0.000000  

### Q4 (20 points)
#### Drop features with excessive NaN

Again, using the quality report summary table that is returned from the function you wrote for Q2, we are going to continue investigating our data. For this, you should still be using the table returned for the [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv) and not the toy dataset I provided for testing.

Like the features that were dropped for Q3, some of the extracted features don't work on all of the variates of the input multi-variate time series samples very well.  So, some of these features return an excessive number of `NaN` values.  These are not verry useful features, so we want to get rid of them before we continue. To do this, you are to write a function that takes in the summary table and the input dataset DataFrame, and drops the features that have **more than 1%** of the entries as null/nan values. Again, these features should be dropped from both the data quality report summary table and from the actual input dataset DataFrame.

As in Q3, a useful method for this operation is:

* [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) (Make sure to use the inplace option otherwise it returns a copy)

In [603]:
def drop_excessive_nan_data(summary_table:DataFrame, data:DataFrame) -> None:
    """
    a function that takes in the summary table and the input dataset DataFrame, and drops 
    the features that have more than 1% of the entries as null/nan values. 
    
    Parameters
    ----------
    summary_table : DataFrame
        a summary table that contains extracted pieces of a data quality report for the partnering analytic base table
        
    data : DataFrame
        an analytical base table dataframe or general dataframe used alongside the quality report to investigate the data
    Output
    ----------
    returns a summary table dataframe after removing features that have have more than 1% of the entries as null/nan values r from the dataframe.
    """
    
    Total_Nulls = summary_table['Null Count']
    Total_NonNulls = summary_table['Non-null Count']
    Name = summary_table['Feature Name']
 
    NamesToDrop = []
    for Names,Null,NonNull in zip(Name, Total_Nulls,Total_NonNulls):
        
        totalPercen =Null/(Null +NonNull)
        if totalPercen >.01:
            NamesToDrop.append(Names)
            
 
    
    summary_table = summary_table[~summary_table['Feature Name'].isin(NamesToDrop)]
    
    for Names in NamesToDrop:
        abt.drop(Names, axis=1, inplace=True)
    
    
    
            
            
        
        
   
    
 
    
    pass
drop_excessive_nan_data(summary_table, abt)

In [604]:
#drop_excessive_nan_data(summary_table, abt)
print(summary_table.shape)

(801, 13)


Now that you have the data cleaned up a little, save the results of both your summary table and your analytics base table using the [pandas.to_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) method. We will want to use these results for the next homework assignment.

In [605]:
out_dir = '/Users/carltonbrown/Downloads/partition1/'
out_summary_table_name = 'data_quality_table.csv'
out_abt_name = 'cleaned_partition1ExtractedFeatures.csv'

In [606]:
summary_table.to_csv(os.path.join(out_dir, out_summary_table_name))
abt.to_csv(os.path.join(out_dir,out_abt_name))

### Q5 (20 points)
#### Add docstring to your functions

Let's revisit our programming skill while learning Fundamentals of Data Science. 

Your code is only as valuable as its reusability. Without understandable and legible documentation (which makes maintainability and reusability possible) nobody would like to use your code, let alone to pay for it. ;)

If you want to know more about the value of documentation, read [this article](https://www.freecodecamp.org/news/why-documentation-matters-and-why-you-should-include-it-in-your-code-41ef62dd5c2f/). There are even conferences on this topic; see [this website](https://www.writethedocs.org/guide/writing/beginners-guide-to-docs/).

In Python, the documentation that is embedded in the code is called **docstring**. In the example below, the "string" wrapped in triple quotes is there to tell us all about this function.

In [607]:
def nanmean(a, axis=None):
    """
    Compute the arithmetic mean along the specified axis, ignoring NaNs.
    
Parameters
    ----------
    a : array_like
        Array containing numbers whose mean is desired. If `a` is not an
        array, a conversion is attempted.
    axis : {int, tuple of int, None}, optional
        Axis or axes along which the means are computed. The default is to compute
        the mean of the flattened array.
    """
    # some magic happens here that we don't care about.
    pass


Note that this is not just a *comment*. If you execute `nanmean` and then call it (as if you want to use it), you can hit `shift+Tab` while your cursor is on the function name, and see how the docstring gets compiled and then pops up. This allows other users to see our description even when they don't have access to our source code. Try it! You can do this with other NumPy and Pandas functions/methods that you've been using.

The above example is a simplified version of the method `nanmean` copied from the NumPy library ([here](https://github.com/numpy/numpy/blob/v1.21.0/numpy/lib/nanfunctions.py#L862-L957)). Feel free to check out their complete docstrings.


Your last task is to provide docstrings for the 4 methods you've implemented. Simply go back to those cells and modify your functions. Feel free to use the text provided to you (in the assignment descriptions) to enrich your docstrings. Keep in mind that your docstring needs (1) a general description, (2) a short description for each input, and (3) a short description for the output.

How to check your docstring? Hit `shift+Tab` and see if the pop-up message is correctly compiled, and make sure your description answers all the questions about your functions.